In [8]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

# Function to apply heatmap to an image
def apply_heatmap(image_path, output_path):
    # Read the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Normalize the image to the range 0-255
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)

    # Apply the heatmap
    heatmap = cv2.applyColorMap(img, cv2.COLORMAP_JET)
    
    # Save the heatmap image
    cv2.imwrite(output_path, heatmap)

# Path to the folder containing raw images
input_folder = 'D:/GitProjects/Datasets/ImageData/valid/non_autistic'
# Path to the folder to save heatmap images
output_folder = 'D:/GitProjects/Datasets/ImageData/heatmaps/valid/non_autistic'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Process each image in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)
        
        apply_heatmap(input_path, output_path)

print("Heatmap images have been saved to:", output_folder)


Heatmap images have been saved to: D:/GitProjects/Datasets/ImageData/heatmaps/valid/non_autistic


## Loading Required Libraries

In [18]:
import os
import shutil
import splitfolders
import tensorflow as tf
import seaborn as sns
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("float32")
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Dense,Activation,Dropout,Conv2D,MaxPooling2D,BatchNormalization,Flatten
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, load_model, Sequential
from sklearn.metrics import confusion_matrix, classification_report
from mlxtend.plotting import plot_confusion_matrix
import itertools
import matplotlib.pyplot as plt
import datetime
import matplotlib.image as mping
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd

## Model Definition

In [19]:
# Function to create a model
def create_model(model_name):
    input_shape = (224, 224, 3)
    base_model = None
    
    #if model_name == 'EfficientNetB4':
    base_model = EfficientNetB4(include_top=False, weights="imagenet", input_shape=input_shape, pooling='max')
    #elif model_name == 'VGG19':
    #    base_model = VGG19(include_top=False, weights="imagenet", input_shape=input_shape, pooling='max')
    #elif model_name == 'NASNetMobile':
    #    base_model = NASNetMobile(include_top=False, weights="imagenet", input_shape=input_shape, pooling='max')
    #elif model_name == 'Xception':
    #    base_model = Xception(include_top=False, weights="imagenet", input_shape=input_shape, pooling='max')

    x = base_model.output
    x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.002)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(rate=0.45, seed=42)(x)
    output = Dense(class_count, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(tf.keras.optimizers.Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

## Train & Evaluate Function

In [20]:
# Function to train and evaluate a model
def train_and_evaluate_model(model, train_data, val_data, test_data, model_name):
    early_stopper = tf.keras.callbacks.EarlyStopping(patience=5, min_delta=0.01, verbose=1)
    history = model.fit(train_data, epochs=60, validation_data=val_data, callbacks=[early_stopper])

    test_loss, test_accuracy = model.evaluate(test_data)
    print(f"Test Accuracy for {model_name}:", test_accuracy)

    test_data_array = []
    labels_array = []
    for images, labels in test_data:
        test_data_array.append(images.numpy())
        labels_array.append(labels.numpy())

    X_test = np.concatenate(test_data_array, axis=0)
    y_test = np.concatenate(labels_array, axis=0)

    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    y_pred = tf.keras.utils.to_categorical(y_pred, num_classes=2)

    print(classification_report(y_test, y_pred))

    y_pred = np.argmax(y_pred, axis=1)
    y_test = np.argmax(y_test, axis=1)

    cm = confusion_matrix(y_test, y_pred)

    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, cmap='crest', annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix for {model_name}')
    plt.show()

## Output

In [24]:
# Path to the consolidated data
loc = "D:/GitProjects/Datasets/ImageData/consolidated"
output_loc = "D:/GitProjects/Datasets/ImageData/"

# Create output directories
os.makedirs(os.path.join(output_loc, 'output'), exist_ok=True)
os.makedirs(os.path.join(output_loc, 'output/train'), exist_ok=True)
os.makedirs(os.path.join(output_loc, 'output/val'), exist_ok=True)
os.makedirs(os.path.join(output_loc, 'output/test'), exist_ok=True)

# Split data into training, validation, and test sets
splitfolders.ratio(loc, output=os.path.join(output_loc, 'output'), seed=42, ratio=(0.80, 0.1, 0.1))

# Load data
train_dir = os.path.join(output_loc, "output/train")
test_dir = os.path.join(output_loc, "output/test")
val_dir = os.path.join(output_loc, "output/val")

train_data = image_dataset_from_directory(train_dir, batch_size=32, image_size=(224, 224), label_mode='categorical', shuffle=True, seed=42)
test_data = image_dataset_from_directory(test_dir, batch_size=32, image_size=(224, 224), label_mode='categorical', shuffle=False, seed=42)
val_data = image_dataset_from_directory(val_dir, batch_size=32, image_size=(224, 224), label_mode='categorical', shuffle=False, seed=42)

class_names = train_data.class_names
class_count = len(class_names)

# Train and evaluate models
model_names = ['EfficientNetB4']

Copying files: 2936 files [00:01, 2411.88 files/s]


Found 2348 files belonging to 2 classes.
Found 296 files belonging to 2 classes.
Found 292 files belonging to 2 classes.


## Main

In [ ]:
for model_name in model_names:
    model = create_model(model_name)
    train_and_evaluate_model(model, train_data, val_data, test_data, model_name)